# Exporting ATT&CK Group Navigator Layers

# Get Relationship STIX Objects - (Manual)
-----------------------
I believe it is important to understand the code behind the main functions available in the Python library [attackcti](https://attackcti.readthedocs.io/en/latest/index.html). I highly recommend to first read the docs I put together about [cti-taxii-client](https://attackcti.readthedocs.io/en/latest/taxii_client.html) and [cti-python-stix2](https://attackcti.readthedocs.io/en/latest/stix.html) libraries.
Those two summarize several of the concepts that I had to read to understand how to perform a simple query against ATT&CK's TAXII server

## Import STIX and TAXII Libraries

In [1]:
from stix2 import TAXIICollectionSource, Filter, CompositeDataSource
from taxii2client.v20 import Collection

## Set ATT&CK TAXII Collection ID Variables
The public ATT&CK TAXII instance has three main collections (Enterprise, Pre and Mobile). Every collection has an ID which attackcti uses to retrieve ATT&CK STIX objects from all those matrices.

In [2]:
ATTACK_STIX_COLLECTIONS = "https://cti-taxii.mitre.org/stix/collections/"
ENTERPRISE_ATTACK = "95ecc380-afe9-11e4-9b6c-751b66dd541e"
PRE_ATTACK = "062767bd-02d2-4b72-84ba-56caef0f8658"
MOBILE_ATTACK = "2f669986-b40b-4423-b720-4396ca6a462b"
ICS_ATTACK = "02c3ef24-9cd4-48f3-a99f-b74ce24f1d34"

## Initialize TAXII Collection Sources
According to [STIX2 docs](https://stix2.readthedocs.io/en/latest/index.html), the [TAXIICollectionSource API](https://stix2.readthedocs.io/en/latest/api/datastore/stix2.datastore.taxii.html#stix2.datastore.taxii.TAXIICollectionSource) provides an interface for searching/retrieving STIX objects from a local/remote TAXII Collection endpoint. In our case, we are pointing to our ATT&CK TAXII Collection instances (https://cti-taxii.mitre.org/stix/collections/<Collection ID>)

In [3]:
ENTERPRISE_COLLECTION = Collection(ATTACK_STIX_COLLECTIONS + ENTERPRISE_ATTACK + "/")
TC_ENTERPRISE_SOURCE = TAXIICollectionSource(ENTERPRISE_COLLECTION)
PRE_COLLECTION = Collection(ATTACK_STIX_COLLECTIONS + PRE_ATTACK + "/")
TC_PRE_SOURCE = TAXIICollectionSource(PRE_COLLECTION)
MOBILE_COLLECTION = Collection(ATTACK_STIX_COLLECTIONS + MOBILE_ATTACK + "/")
TC_MOBILE_SOURCE = TAXIICollectionSource(MOBILE_COLLECTION)
ICS_COLLECTION = Collection(ATTACK_STIX_COLLECTIONS + ICS_ATTACK + "/")
TC_ICS_SOURCE = TAXIICollectionSource(ICS_COLLECTION)

## Initialize a Composite Data Source
According to [STIX2 docs](https://stix2.readthedocs.io/en/latest/index.html), a user can have a single [CompositeDataSource](https://stix2.readthedocs.io/en/latest/api/stix2.datastore.html#stix2.datastore.CompositeDataSource) as an interface to a set of DataSources. When an API call is made to the CompositeDataSource, it is delegated to each of the (real) DataSources that are attached to it. In our case, we have three TAXIICollection sources (Enterprise, PRE and Mobile) as defined in our previous step. Therefore, we can use the CompositeDataSource class and the add_data_sources method to attach every ATT&CK TAXIICollection source and be able to query all of them at the same time.

In [4]:
COMPOSITE_DS = CompositeDataSource()
COMPOSITE_DS.add_data_sources([TC_ENTERPRISE_SOURCE, TC_PRE_SOURCE, TC_MOBILE_SOURCE, TC_ICS_SOURCE])

## Retrieve all relationships
Now that we can query all the ATT&CK TAXIICollection sources at once, we can use the query method and a set of filters to retrieve STIX objects of type relationship

In [5]:
rels = COMPOSITE_DS.query(Filter("type", "=", "relationship"))
rels[0]

Relationship(type='relationship', id='relationship--fcee0cef-7d5b-49da-928c-2a3d0cfd06b0', created_by_ref='identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5', created='2020-11-10T18:04:03.668Z', modified='2020-11-10T18:04:03.668Z', relationship_type='uses', description="(Citation: FireEye KEGTAP SINGLEMALT October 2020)(Citation: DHS/CISA Ransomware Targeting Healthcare October 2020)(Citation: DFIR Ryuk's Return October 2020)(Citation: DFIR Ryuk 2 Hour Speed Run November 2020)(Citation: DFIR Ryuk in 5 Hours October 2020)(Citation: Sophos New Ryuk Attack October 2020)", source_ref='intrusion-set--dd2d9ca6-505b-4860-a604-233685b802c7', target_ref='malware--a7881f21-e978-4fe4-af56-92c9416a2616', external_references=[ExternalReference(source_name='FireEye KEGTAP SINGLEMALT October 2020', description='Kimberly Goody, Jeremy Kennelly, Joshua Shilko, Steve Elovitz, Douglas Bienstock. (2020, October 28). Unhappy Hour Special: KEGTAP and SINGLEMALT With a Ransomware Chaser. Retrieved October 28, 2

## Retrieve all relationships from an specific STIX object
What if you want to be very specific and get relationships from a specific STIX objects? You can use the [relationships](https://stix2.readthedocs.io/en/latest/api/stix2.datastore.html#stix2.datastore.CompositeDataSource.relationships) method from the [CompositeDataSource](https://stix2.readthedocs.io/en/latest/api/stix2.datastore.html#stix2.datastore.CompositeDataSource) class to retrieve relationships involving a given STIX object.

In [6]:
from attackcti import attack_client
lift = attack_client()

groups = lift.get_groups()
groups = lift.remove_revoked(groups)

rels = COMPOSITE_DS.relationships(groups[0], 'uses', source_only=True)
rels[0]

Relationship(type='relationship', id='relationship--689b0bff-7eb4-4678-997b-64794c56add0', created_by_ref='identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5', created='2020-09-22T20:17:38.809Z', modified='2020-10-06T15:32:20.360Z', relationship_type='uses', description='[GOLD SOUTHFIELD](https://attack.mitre.org/groups/G0115) has distributed ransomware by backdooring software installers via a strategic web compromise of the site hosting Italian WinRAR.(Citation: Secureworks REvil September 2019)(Citation: Secureworks GandCrab and REvil September 2019)(Citation: Secureworks GOLD SOUTHFIELD)', source_ref='intrusion-set--c77c5576-ca19-42ed-a36f-4b4486a84133', target_ref='attack-pattern--bd369cd9-abb8-41ce-b5bb-fff23ee86c00', external_references=[ExternalReference(source_name='Secureworks REvil September 2019', description='Counter Threat Unit Research Team. (2019, September 24). REvil/Sodinokibi Ransomware. Retrieved August 4, 2020.', url='https://www.secureworks.com/research/revil-sodinokib

# Get Relationship STIX Objects - (Automatic)
-----------------------

## Retrieve all relationships

In [7]:
from attackcti import attack_client
lift = attack_client()

In [8]:
%time all_relationships = lift.get_relationships()

CPU times: user 2.64 s, sys: 71 ms, total: 2.71 s
Wall time: 4.36 s


In [9]:
all_relationships[0]

Relationship(type='relationship', id='relationship--fcee0cef-7d5b-49da-928c-2a3d0cfd06b0', created_by_ref='identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5', created='2020-11-10T18:04:03.668Z', modified='2020-11-10T18:04:03.668Z', relationship_type='uses', description="(Citation: FireEye KEGTAP SINGLEMALT October 2020)(Citation: DHS/CISA Ransomware Targeting Healthcare October 2020)(Citation: DFIR Ryuk's Return October 2020)(Citation: DFIR Ryuk 2 Hour Speed Run November 2020)(Citation: DFIR Ryuk in 5 Hours October 2020)(Citation: Sophos New Ryuk Attack October 2020)", source_ref='intrusion-set--dd2d9ca6-505b-4860-a604-233685b802c7', target_ref='malware--a7881f21-e978-4fe4-af56-92c9416a2616', external_references=[ExternalReference(source_name='FireEye KEGTAP SINGLEMALT October 2020', description='Kimberly Goody, Jeremy Kennelly, Joshua Shilko, Steve Elovitz, Douglas Bienstock. (2020, October 28). Unhappy Hour Special: KEGTAP and SINGLEMALT With a Ransomware Chaser. Retrieved October 28, 2

## Retrieve all relationships from an specific STIX object

In [10]:
groups = lift.get_groups()
groups = lift.remove_revoked(groups)

In [11]:
%time group_relationships = lift.get_relationships_by_object(groups[0])

CPU times: user 274 ms, sys: 35.5 ms, total: 310 ms
Wall time: 1.95 s


In [12]:
group_relationships[0]

Relationship(type='relationship', id='relationship--689b0bff-7eb4-4678-997b-64794c56add0', created_by_ref='identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5', created='2020-09-22T20:17:38.809Z', modified='2020-10-06T15:32:20.360Z', relationship_type='uses', description='[GOLD SOUTHFIELD](https://attack.mitre.org/groups/G0115) has distributed ransomware by backdooring software installers via a strategic web compromise of the site hosting Italian WinRAR.(Citation: Secureworks REvil September 2019)(Citation: Secureworks GandCrab and REvil September 2019)(Citation: Secureworks GOLD SOUTHFIELD)', source_ref='intrusion-set--c77c5576-ca19-42ed-a36f-4b4486a84133', target_ref='attack-pattern--bd369cd9-abb8-41ce-b5bb-fff23ee86c00', external_references=[ExternalReference(source_name='Secureworks REvil September 2019', description='Counter Threat Unit Research Team. (2019, September 24). REvil/Sodinokibi Ransomware. Retrieved August 4, 2020.', url='https://www.secureworks.com/research/revil-sodinokib

# Retrive Techniques used by one Group - (Manual)
-----------------------

In this case we want relationship objects that have target_ref values of type attack-pattern. Following the manual code I shared above, and the results from the `get_relationships_by_object()` function, you can simply query the ATT&CK Enterprise TAXIICollection source with the filter below

In [13]:
filter_objects = [
  Filter('type', '=', 'attack-pattern'),
  Filter('id', '=', [r.target_ref for r in group_relationships])
]
techniques_used = TC_ENTERPRISE_SOURCE.query(filter_objects)
techniques_used[0]

AttackPattern(type='attack-pattern', id='attack-pattern--bd369cd9-abb8-41ce-b5bb-fff23ee86c00', created_by_ref='identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5', created='2020-03-11T14:17:21.153Z', modified='2020-03-11T14:17:21.153Z', name='Compromise Software Supply Chain', description='Adversaries may manipulate application software prior to receipt by a final consumer for the purpose of data or system compromise. Supply chain compromise of software can take place in a number of ways, including manipulation of the application source code, manipulation of the update/distribution mechanism for that software, or replacing compiled releases with a modified version.\n\nTargeting may be specific to a desired victim set or may be distributed to a broad set of consumers but only move on to additional tactics on specific victims.(Citation: Avast CCleaner3 2018) (Citation: Command Five SK 2011)  ', kill_chain_phases=[KillChainPhase(kill_chain_name='mitre-attack', phase_name='initial-access')], 

# Retrive Techniques used by one Group - (Automatic)
-----------------------

In [14]:
from attackcti import attack_client
lift = attack_client()
groups = lift.get_groups()
groups = lift.remove_revoked(groups)
group_techniques = lift.get_techniques_used_by_group(groups[0])
group_techniques[0]

AttackPattern(type='attack-pattern', id='attack-pattern--bd369cd9-abb8-41ce-b5bb-fff23ee86c00', created_by_ref='identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5', created='2020-03-11T14:17:21.153Z', modified='2020-03-11T14:17:21.153Z', name='Compromise Software Supply Chain', description='Adversaries may manipulate application software prior to receipt by a final consumer for the purpose of data or system compromise. Supply chain compromise of software can take place in a number of ways, including manipulation of the application source code, manipulation of the update/distribution mechanism for that software, or replacing compiled releases with a modified version.\n\nTargeting may be specific to a desired victim set or may be distributed to a broad set of consumers but only move on to additional tactics on specific victims.(Citation: Avast CCleaner3 2018) (Citation: Command Five SK 2011)  ', kill_chain_phases=[KillChainPhase(kill_chain_name='mitre-attack', phase_name='initial-access')], 

# Retrive all Techniques used by all Groups - (Manual)
-----------------------
You can apply the same get_techniques_used_by_group() function, but against all the groups STIX objects that the get_groups() function retrieves. You can do a simple for loop over more than 90 groups. However, it takes longer than what I would like it to take. Therefore, I decided to go a different route and started testing some code.

## Get all groups and techniques

In [15]:
from attackcti import attack_client
lift = attack_client()
groups = lift.get_groups()
techniques = lift.get_techniques()
techniques = lift.remove_revoked(techniques)

## Filter Group objects using techniques

In [16]:
from stix2.utils import get_type_from_id
group_relationships = []
for rel in all_relationships:
    if get_type_from_id(rel.source_ref) == 'intrusion-set'\
    and get_type_from_id(rel.target_ref) == 'attack-pattern':
        group_relationships.append(rel)
len(group_relationships)
print(group_relationships[0])

{
    "type": "relationship",
    "id": "relationship--82dee5a5-7890-4bed-bc8c-83ffa13a8bcf",
    "created_by_ref": "identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5",
    "created": "2020-11-10T17:28:19.540Z",
    "modified": "2020-11-10T17:28:19.540Z",
    "relationship_type": "uses",
    "description": "[Wizard Spider](https://attack.mitre.org/groups/G0102) has identified domain admins through the use of \u201cnet group \u2018Domain admins\u2019\u201d commands.(Citation: DFIR Ryuk's Return October 2020)",
    "source_ref": "intrusion-set--dd2d9ca6-505b-4860-a604-233685b802c7",
    "target_ref": "attack-pattern--21875073-b0ee-49e3-9077-1e2a885359af",
    "external_references": [
        {
            "source_name": "DFIR Ryuk's Return October 2020",
            "description": "The DFIR Report. (2020, October 8). Ryuk\u2019s Return. Retrieved October 9, 2020.",
            "url": "https://thedfirreport.com/2020/10/08/ryuks-return/"
        }
    ],
    "object_marking_refs": [
        "

## Match Group -> Relationships Intrusion-set ID
Then, I just take all the group_relationships I got, and look for the specific `intrusion-set (Group)` id in the groups STIX objects. Once there is a match, I create new fields on the `intrusion-set (Group)` STIX object to add additional information about the `attack-pattern (Technique)` in the relationship object. The most important additional metadata is the target_ref field which points to the specific `attack-pattern (Technique)` id involving the `group`. The results are then added to a new list named `group_techniques_ref` .

In [17]:
import json
group_techniques_ref = []
for g in groups:
    for rel in group_relationships:
        if g['id'] == rel['source_ref']:
            gs = json.loads(g.serialize())
            gs
            gs['technique_ref'] = rel['target_ref']
            gs['relationship_description'] = rel['description']
            gs['relationship_id'] = rel['id']
            group_techniques_ref.append(gs)

## Match Attack-patterns -> Intrusion-set object ID
I apply the same concept as before, and just loop through all the attack-pattern objects and look for the specific attack-pattern id in the initial relationships STIX objects. Once there is a match, I add additional information from the attack-pattern itself to the python dictionaries in the `group_techniques_ref` list. The results then get added to a new list named `groups_use_techniques`.

In [18]:
groups_use_techniques = []
for gt in group_techniques_ref:
    for t in techniques:
        if gt['technique_ref'] == t['id']:
            tactic_list = list()
            for phase in t['kill_chain_phases']:
                tactic_list.append(phase['phase_name'])
                gt['technique'] = t['name']
                gt['technique_description'] = t['description']
                gt['tactic'] = tactic_list
                gt['technique_id'] = t['external_references'][0]['external_id']
                gt['matrix'] =  t['external_references'][0]['source_name']
                if 'x_mitre_platforms' in t.keys():
                    gt['platform'] = t['x_mitre_platforms']
                if 'x_mitre_data_sources' in t.keys():
                    gt['data_sources'] = t['x_mitre_data_sources']
                if 'x_mitre_permissions_required' in t.keys():
                    gt['permissions_required'] = t['x_mitre_permissions_required']
                if 'x_mitre_effective_permissions' in t.keys():
                    gt['effective_permissions'] = t['x_mitre_effective_permissions']
                groups_use_techniques.append(gt)
groups_use_techniques[0]

{'created_by_ref': 'identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5',
 'object_marking_refs': ['marking-definition--fa42a846-8d90-4e51-bc29-71d5b4802168'],
 'external_references': [{'external_id': 'G0115',
   'source_name': 'mitre-attack',
   'url': 'https://attack.mitre.org/groups/G0115'},
  {'source_name': 'Secureworks REvil September 2019',
   'url': 'https://www.secureworks.com/research/revil-sodinokibi-ransomware',
   'description': 'Counter Threat Unit Research Team. (2019, September 24). REvil/Sodinokibi Ransomware. Retrieved August 4, 2020.'},
  {'source_name': 'Secureworks GandCrab and REvil September 2019',
   'url': 'https://www.secureworks.com/blog/revil-the-gandcrab-connection',
   'description': 'Secureworks . (2019, September 24). REvil: The GandCrab Connection. Retrieved August 4, 2020.'},
  {'source_name': 'Secureworks GOLD SOUTHFIELD',
   'url': 'https://www.secureworks.com/research/threat-profiles/gold-southfield',
   'description': 'Secureworks. (n.d.). GOLD SOUTHFIE

# Retrive all Techniques used by all Groups - (Automatic)
-----------------------

In [19]:
from attackcti import attack_client
lift = attack_client()
%time techniques_used = lift.get_techniques_used_by_all_groups()

CPU times: user 5.41 s, sys: 125 ms, total: 5.54 s
Wall time: 8.17 s


In [20]:
len(techniques_used)

1937

In [21]:
techniques_used[0]

{'created_by_ref': 'identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5',
 'object_marking_refs': ['marking-definition--fa42a846-8d90-4e51-bc29-71d5b4802168'],
 'external_references': [{'external_id': 'G0115',
   'source_name': 'mitre-attack',
   'url': 'https://attack.mitre.org/groups/G0115'},
  {'source_name': 'Secureworks REvil September 2019',
   'url': 'https://www.secureworks.com/research/revil-sodinokibi-ransomware',
   'description': 'Counter Threat Unit Research Team. (2019, September 24). REvil/Sodinokibi Ransomware. Retrieved August 4, 2020.'},
  {'source_name': 'Secureworks GandCrab and REvil September 2019',
   'url': 'https://www.secureworks.com/blog/revil-the-gandcrab-connection',
   'description': 'Secureworks . (2019, September 24). REvil: The GandCrab Connection. Retrieved August 4, 2020.'},
  {'source_name': 'Secureworks GOLD SOUTHFIELD',
   'url': 'https://www.secureworks.com/research/threat-profiles/gold-southfield',
   'description': 'Secureworks. (n.d.). GOLD SOUTHFIE

# Create Navigator Group Layer Files - (Manual)
-----------------------

## Create a list of group dictionaries
To make things easier, I create a list of dictionaries where each group name is the main key and the value is a list where I append every single technique involving that group. I get that information from the `get_techniques_used_by_all_groups()` results.

In [22]:
groups = lift.get_groups()
groups = lift.remove_revoked(groups)
groups_list = []
for g in groups:
    group_dict = dict()
    group_dict[g['name']] = []
    groups_list.append(group_dict)
groups_list[89]

{'Naikon': []}

## Group techniques by group
We can then use the output of the `get_techniques_used_by_all_groups()` function and start appending techniques to the dictionaries with the key name that matches the group name being involved with each technique.

In [23]:
for group in groups_list:
    for group_name,techniques_list in group.items():
        for gut in techniques_used:
            if group_name == gut['name']:
                technique_dict = dict()
                technique_dict['techniqueId'] = gut['technique_id']
                technique_dict['techniqueName'] = gut['technique']
                technique_dict['comment'] = gut['relationship_description']
                technique_dict['tactic'] = gut['tactic']
                technique_dict['group_id'] = gut['external_references'][0]['external_id']
                techniques_list.append(technique_dict)
groups_list[89]

{'Naikon': [{'techniqueId': 'T1566.001',
   'techniqueName': 'Spearphishing Attachment',
   'comment': '[Naikon](https://attack.mitre.org/groups/G0019) has used malicious e-mail attachments to deliver malware.(Citation: CheckPoint Naikon May 2020)',
   'tactic': [KillChainPhase(kill_chain_name='mitre-attack', phase_name='initial-access')],
   'group_id': 'G0019'},
  {'techniqueId': 'T1204.002',
   'techniqueName': 'Malicious File',
   'comment': '[Naikon](https://attack.mitre.org/groups/G0019) has convinced victims to open malicious attachments to execute malware.(Citation: CheckPoint Naikon May 2020)',
   'tactic': [KillChainPhase(kill_chain_name='mitre-attack', phase_name='execution')],
   'group_id': 'G0019'},
  {'techniqueId': 'T1137.006',
   'techniqueName': 'Add-ins',
   'comment': '[Naikon](https://attack.mitre.org/groups/G0019) has used the RoyalRoad exploit builder to drop a second stage loader, intel.wll, into the Word Startup folder on the compromised host.(Citation: CheckPo

## Run dynamic navigator layer template

In [24]:
import json
for group in groups_list:
    for k,v in group.items():
        if v:
            actor_layer = {
                "description": ("Enterprise techniques used by {0}, ATT&CK group {1} v1.0".format(k,v[0]['group_id'])),
                "name": ("{0} ({1})".format(k,v[0]['group_id'])),
                "domain": "mitre-enterprise",
                "version": "2.2",
                "techniques": [
                    {
                        "score": 1,
                        "techniqueID" : technique['techniqueId'],
                        "techniqueName" : technique['techniqueName'],
                        "comment": technique['comment']
                    } for technique in v
                ],
                "gradient": {
                    "colors": [
                        "#ffffff",
                        "#ff6666"
                    ],
                    "minValue": 0,
                    "maxValue": 1
                },
                "legendItems": [
                    {
                        "label": ("used by {}".format(k)),
                        "color": "#ff6666"
                    }
                ]
            }
            with open(('{0}_{1}.json'.format(k,v[0]['group_id'])), 'w') as f:
                f.write(json.dumps(actor_layer))

In [25]:
! ls *.json

ALLANITE_G1000.json          Inception_G0100.json
APT-C-36_G0099.json          Ke3chang_G0004.json
APT12_G0005.json             Kimsuky_G0094.json
APT16_G0023.json             Lazarus Group_G0032.json
APT17_G0025.json             Leafminer_G0077.json
APT18_G0026.json             Leviathan_G0065.json
APT19_G0073.json             Machete_G0095.json
APT1_G0006.json              Magic Hound_G0059.json
APT28_G0007.json             Moafee_G0002.json
APT29_G0016.json             Mofang_G0103.json
APT30_G0013.json             Molerats_G0021.json
APT32_G0050.json             MuddyWater_G0069.json
APT33_G0064.json             Naikon_G0019.json
APT37_G0067.json             Night Dragon_G0014.json
APT38_G0082.json             OilRig_G0049.json
APT39_G0087.json             Orangeworm_G0071.json
APT3_G0022.json              PLATINUM_G0068.json
APT41_G0096.json             PROMETHIUM_G0056.json
Axiom_G0001.json             Patchwork_G0040.json
BRONZE BUTLER_G0060.json     PittyTiger_G0011.json
BlackO

We can delete all the files with the following command.

In [26]:
! rm *.json

# Create Navigator Group Layer Files - (Automatic)
-----------------------

In [27]:
from attackcti import attack_client
lift = attack_client()

%time lift.export_groups_navigator_layers()

CPU times: user 6.02 s, sys: 181 ms, total: 6.2 s
Wall time: 9.24 s


In [28]:
! ls *.json

ALLANITE_G1000.json          Inception_G0100.json
APT-C-36_G0099.json          Ke3chang_G0004.json
APT12_G0005.json             Kimsuky_G0094.json
APT16_G0023.json             Lazarus Group_G0032.json
APT17_G0025.json             Leafminer_G0077.json
APT18_G0026.json             Leviathan_G0065.json
APT19_G0073.json             Machete_G0095.json
APT1_G0006.json              Magic Hound_G0059.json
APT28_G0007.json             Moafee_G0002.json
APT29_G0016.json             Mofang_G0103.json
APT30_G0013.json             Molerats_G0021.json
APT32_G0050.json             MuddyWater_G0069.json
APT33_G0064.json             Naikon_G0019.json
APT37_G0067.json             Night Dragon_G0014.json
APT38_G0082.json             OilRig_G0049.json
APT39_G0087.json             Orangeworm_G0071.json
APT3_G0022.json              PLATINUM_G0068.json
APT41_G0096.json             PROMETHIUM_G0056.json
Axiom_G0001.json             Patchwork_G0040.json
BRONZE BUTLER_G0060.json     PittyTiger_G0011.json
BlackO

We can delete all the files with the following command.

In [29]:
! rm *.json